In [52]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
def tokenizer(text): 
    # HTML 태그 제거
    text = re.sub('<[^>]*>', '', text)
    
    # 이모티콘 패턴 검색 (":", ";", "=" 기호로 시작하고, 그 뒤에 ")", "(", "D", "P"가 올 수 있음)
    emoticons = re.findall(r'[:;=](?:-)?[)(DP]', text)
    
    # 텍스트에서 모든 특수문자와 숫자를 공백으로 대체하고, 이모티콘은 다시 추가
    # text = re.sub(r'[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', '')
    text = re.sub('[\W]+', ' ', text.lower()) \
            + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding = 'utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int (line[-2])
            yield text, label
#next는 머리행을 넘어가기 위해
next(stream_docs(path='data/movie_data.csv'))

('"at a Saturday matinee in my home town. I went with an older friend (he was about 12) and my mom let me go because she thought the film would be OK (it\'s rated G). I was assaulted by loud music, STRANGE images, no plot and a stubborn refusal to make ANY sense. We left halfway through because we were bored, frustrated and our ears hurt. <br /><br />I saw it 22 years later in a revival theatre. My opinion had changed--it\'s even WORSE! Basically everything I hated about it was still there and the film was VERY 60s...and has dated badly. I got all the little in-jokes...too bad they weren\'t funny. The constant shifts in tone got quickly annoying and there\'s absolutely nothing to get a firm grip on. Some people will love this. I found it frustrating...by the end of the film I felt like throwing something heavy at the screen.<br /><br />Also, all the Monkees songs in this movie SUCK (and I DO like them).<br /><br />For ex-hippies only...or if you\'re stoned. I give this a 1."',
 0)

In [53]:
def get_minibatch(doc_stream, size):
    docs, y = [],[]
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        pass
    return docs, y

In [54]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',
                         n_features = 2**21,
                         preprocessor= None,
                         tokenizer = tokenizer)
clf = SGDClassifier(loss = 'log_loss', random_state = 1, max_iter = 1)
doc_stream = stream_docs(path = 'data/movie_data.csv')

In [55]:
import pyprind
import sys
pbar = pyprind.ProgBar(45, stream = sys.stdout)
classes = np.array([0,1])

for _ in range(45):
    X_train , y_train = get_minibatch(doc_stream , size = 1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes = classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


In [56]:
X_test, y_test = get_minibatch(doc_stream, size = 5000)
X_vec = vect.transform(X_test)
print(f"정확도 : {clf.score(X_vec, y_test):.3f}")

정확도 : 0.873


In [58]:
clf = clf.partial_fit(X_vec, y_test)
print(clf.score(X_vec, y_test))

0.8962
